In [ ]:
import logging, os, re, datetime
from IPython.display import display
from arcgis.gis import GIS
import pandas as pd

''' ********************** SCRIPT CONFIGURATION START ********************** '''

#What is the ID of the Feature Layer you want to Add attachments to?
FeatureLayerId = ''

#What are your ArcGIS Enterprise/ArcGIS Online credentials? This is case sensitive.
PortalUserName = ''
PortalPassword = ''
PortalUrl = ''

#Where are your attachments stored?
attachmentFolder = r''
attachmentCounter = 0
#Set to False if ArcGIS Enterprise cert is not valid
PortalCertVerification = True
#Connect to GIS, and get Feature Layer information
if PortalUserName == '' and PortalPassword == '':
    gis = GIS()
else:
    gis = GIS(PortalUrl, PortalUserName, PortalPassword, verify_cert=PortalCertVerification)

#Display item

itemObject = gis.content.get(FeatureLayerId)
display(itemObject)

# #Loop through layers in Feature Layer
for i in range(len(itemObject.layers)):
    featureLayer = itemObject.layers[i]
 #Skip layer if attachments are not enabled
    if featureLayer.properties.hasAttachments == True:
#Query to get list of all records to start with
        query1 = featureLayer.query(where='1=1', return_all_records='true')
    #Create dataframe object and get list of attributes to match attachments to
        df = query1.df
        df.sort_index()
    else:
        print('Layer does not have Attachments Enabled')
    for index, row in df.iterrows():
        ObjectID = (row['OBJECTID'])
        globalid = (row['GlobalID'])

        featureAttachments = featureLayer.attachments.get_list(oid=ObjectID)
        #check if features have attachments:
        if len(featureAttachments) == 0:
    #   If no attachments exist contineu to Search specified folder for attachments to add
            for root, dirs, files in os.walk(attachmentFolder): 
                #List files in folder
                for filename in files:
                    attachment = re.findall((globalid), filename, re.IGNORECASE)
                    if not attachment: continue               
                    for pic in attachment:                   
                         #find individual globalid's
                        features = df.GlobalID.str.upper() == pic
                        for index, row in df[features].iterrows():
                            realOID = (row['OBJECTID'])
                            filePath = os.path.abspath(os.path.join(root, filename))
                                    #attach the appropriate files
                            featureLayer.attachments.add(realOID, filePath)

                            attachmentCounter += 1
                            print('Added Attachment: {} to ObjectID {}'.format(filename, realOID))
                            print('{} attachments added so far...'.format(attachmentCounter))
        #if features already have attachments                     
        else:
            attachments = featureLayer.attachments.get_list(oid=ObjectID)            
            df2 = pd.DataFrame(attachments)
            attache = []
            for row in df2.itertuples():
                attName = row.name                
                attache.append(attName)                
                continue    
            for root, dirs, files in os.walk(attachmentFolder):             
                for filename in files:
                    if filename in attache:
                        print('Attachment:{} already exists.'.format(filename))
                    else:
                        attachment = re.findall((globalid), filename, re.IGNORECASE)
                        if not attachment: continue               
                        for pic in attachment:
                            features = df.GlobalID.str.upper() == pic
                            for index, row in df[features].iterrows():
                                realOID = (row['OBJECTID'])
                                filePath = os.path.abspath(os.path.join(root, filename))
                                #attach the appropriate files
                                featureLayer.attachments.add(realOID, filePath)
                                attachmentCounter += 1
                                print('Added Attachment: {} to ObjectID {}'.format(filename, realOID))
                                print('{} attachments added so far...'.format(attachmentCounter))    
